<h1> db_score_3_labels.xlsx을 mysql의 테이블로 구축하기

In [498]:
import pymysql
import pandas as pd

def load_dbscore_data():
    xl_file = 'db_score_3_labels.xlsx'
    db_score = pd.read_excel(xl_file)

    conn = pymysql.connect(host='localhost', user='june', password='wnsgur97895', db='datascience')
    curs = conn.cursor(pymysql.cursors.DictCursor)
        
    drop_sql = """drop table if exists db_score """
    curs.execute(drop_sql)
    conn.commit()
    
    import sqlalchemy
    database_username = 'june'
    database_password = 'wnsgur97895'
    database_ip       = 'localhost'
    database_name     = 'datascience'
    database_connection = sqlalchemy.create_engine('mysql+pymysql://{0}:{1}@{2}/{3}'.
                                                   format(database_username, database_password, 
                                                          database_ip, database_name))
    db_score.to_sql(con=database_connection, name='db_score', if_exists='replace')

load_dbscore_data()
print("mysql의 테이블로 구축 완료")

mysql의 테이블로 구축 완료


<h1>grade B에 대한 SVM binary classification

In [522]:
import pymysql
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def classification_performance_eval(y, y_predict):
    tp, tn, fp, fn = 0,0,0,0
    
    for y, yp in zip(y,y_predict):
        if y == 1 and yp == 1:
            tp += 1
        elif y == 1 and yp == -1:
            fn += 1
        elif y == -1 and yp == 1:
            fp += 1
        else:
            tn += 1
    #print("===============================")        
    #print("tp : " + str(tp))
    #print("tn : " + str(tn))        
    #print("fp : " + str(fp))        
    #print("fn : " + str(fn))        
    #print("===============================")        
        
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    
    precision = (tp)/(tp+fp) if ((tp+fp) != 0) else 0
    recall = (tp)/(tp+fn) if ((tp+fn) != 0) else 0
    f1_score = 2*precision*recall / (precision+recall) if ((precision+recall) != 0) else 0
    
    return accuracy, precision, recall, f1_score


conn = pymysql.connect(host='localhost', user='june', password='wnsgur97895', db='datascience')
curs = conn.cursor(pymysql.cursors.DictCursor)

sql = "select * from db_score"
curs.execute(sql)

data  = curs.fetchall()

#print(data)

curs.close()
conn.close()

#db_score의 feature variable
X = [ (t['homework'], t['discussion'], t['final']) for t in data ]
X = np.array(X)

#db_score의 label attribute
y = [ 1 if (t['grade'] == 'B') else -1 for t in data ]
y = np.array(y)


#train_test_split의 경우
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.26, random_state=42)

#SVM import

from sklearn.svm import SVC

kernelName = 'rbf'

cNum1 =1e5

svm = SVC(kernel = kernelName, C= int(cNum1))

svm = svm.fit(X_train, y_train)

y_predict = svm.predict(X_test)

acc, prec, rec, f1 = classification_performance_eval(y_test, y_predict)

#print(X_test)

print("====================SVM binary classification(kernel ='"+kernelName+"', C="+str(cNum1)+") train_test_split의 경우===================")
print("train데이터 개수 : "+str(len(X_train)) )
print("test데이터 개수 : "+str(len(X_test)) )
print("accuracy=%f" %acc)
print("precision=%f" %prec)
print("recall=%f" %rec)
print("f1_score=%f" %f1)

#K-fold cross validation의 경우
from sklearn.model_selection import KFold

accuracy = []
precision = []
recall = []
f1_score = []

kNum2 = 15   #데이트를 kNum 등분해서 교차검증
cNum2 = 1e2  #SVC C값

kf = KFold(n_splits= int(kNum2), random_state=42, shuffle=True)

for train_index, test_index in kf.split(X):
    X_train2, X_test2 = X[train_index], X[test_index]
    y_train2, y_test2 = y[train_index], y[test_index]
    
    svm2 = SVC(kernel = 'rbf' , C=int(cNum2))
    svm2 = svm2.fit(X_train2, y_train2)
    y_predict2 = svm2.predict(X_test2)
    acc, prec, rec, f1 = classification_performance_eval(y_test2, y_predict2)

    accuracy.append(acc)
    precision.append(prec)
    recall.append(rec)
    f1_score.append(f1)


   
import statistics
print()
print("====================SVM binary classification(kernal = 'rbf', C="+str(cNum2)+") "+str(kNum2)+"-fold cross validation의 경우================")

print("average_accuracy =%f"%statistics.mean(accuracy))
print("average_precision =%f"%statistics.mean(precision))
print("average_recall =%f"%statistics.mean(recall))
print("average_f1_score =%f"%statistics.mean(f1_score))

#from sklearn.metrics import classification_report
#print(classification_report(y_test, y_predict))


====================SVM binary classification(kernel ='rbf', C=100000.0) train_test_split의 경우===================
train데이터 개수 : 68
test데이터 개수 : 24
accuracy=0.833333
precision=0.666667
recall=0.857143
f1_score=0.750000

====================SVM binary classification(kernal = 'rbf', C=100.0) 15-fold cross validation의 경우================
average_accuracy =0.695238
average_precision =0.527778
average_recall =0.700000
average_f1_score =0.568571


<h1>grade B에 대한 Logistic Regression 알고리즘의 binary classification

In [523]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

#db_score의 feature variable
X = [ (t['homework'], t['discussion'], t['final']) for t in data ]
X = np.array(X)

#db_score의 label attribute
y = [ 1 if (t['grade'] == 'B') else -1 for t in data ]
y = np.array(y)

#scaler = StandardScaler()

#X = scaler.fit_transform(X)
#train_test_split의 경우
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.12, random_state=42)

#스케일러 생성 => 각 feature들의 스케일을 맞추어 줘야함
#각 feature의 평균을 0, 표준편차를 1로 맞춰줌

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

cNum3 = 1

lr = LogisticRegression(C= int(cNum3),class_weight='balanced', random_state=42)

lr.fit(X_train_scaled, y_train)

lr_y_predict = lr.predict(X_test_scaled)

lr_acc, lr_prec, lr_rec, lr_f1 = classification_performance_eval(y_test, lr_y_predict)

print("====================Logistic Regression binary classification(C="+str(cNum3)+") train_test_split의 경우===================")
print("train데이터 개수 : "+str(len(X_train)) )
print("test데이터 개수 : "+str(len(X_test_scaled)) )
print("accuracy=%f" %lr_acc)
print("precision=%f" %lr_prec)
print("recall=%f" %lr_rec)
print("f1_score=%f" %lr_f1)

#print('테스트 셋 관측 수 : ' + str(len(y)) )
#print('오분류 관측 수 : ', sum(lr_y_predict != y_test))

#K-fold cross validation의 경우
from sklearn.model_selection import KFold

l_accuracy = []
l_precision = []
l_recall = []
l_f1_score = []

kNum3 = 23  #데이트를 kNum 등분해서 교차검증
cNum3 = 1 #SVC C값

kf = KFold(n_splits= int(kNum3), random_state=42, shuffle=True)

for train_index, test_index in kf.split(X):
    X_train2, X_test2 = X[train_index], X[test_index]
    y_train2, y_test2 = y[train_index], y[test_index]
    
    X_train2_scaled = scaler.fit_transform(X_train2)
    X_test2_scaled = scaler.fit_transform(X_test2)
    
    lr = LogisticRegression(C= int(cNum3), class_weight='balanced', random_state=42)
    lr.fit(X_train2_scaled, y_train2)
    lr_y_predict = lr.predict(X_test2_scaled)
    lr_acc, lr_prec, lr_rec, lr_f1 = classification_performance_eval(y_test, lr_y_predict)

    l_accuracy.append(lr_acc)
    l_precision.append(lr_prec)
    l_recall.append(lr_rec)
    l_f1_score.append(lr_f1)
    
import statistics


print("====================Logistic Regression binary classification(C="+str(cNum3)+") "+str(kNum3)+"-fold cross validation의 경우================")
print("average_accuracy =%f"%statistics.mean(l_accuracy))
print("average_precision =%f"%statistics.mean(l_precision))
print("average_recall =%f"%statistics.mean(l_recall))
print("average_f1_score =%f"%statistics.mean(l_f1_score))



====================Logistic Regression binary classification(C=1) train_test_split의 경우===================
train데이터 개수 : 80
test데이터 개수 : 12
accuracy=0.583333
precision=0.500000
recall=0.800000
f1_score=0.615385
====================Logistic Regression binary classification(C=1) 23-fold cross validation의 경우================
average_accuracy =0.673913
average_precision =0.666667
average_recall =0.695652
average_f1_score =0.666667


<h1>grade A,B,C에 대한 SVM multi-class classification

In [524]:
def multi_classification_performance_eval(y, y_predict):
    
    arr = np.zeros((3,3)) #3x3배열 생성
    
    for y, yp in zip(y,y_predict):
        if y == 0 and yp == 0:
            arr[0,0] += 1
        elif y == 0 and yp == 1:
            arr[1,0] += 1
        elif y == 0 and yp == 2:
            arr[2,0] += 1
        elif y == 1 and yp == 0: 
            arr[0,1] += 1
        elif y == 1 and yp == 1:
            arr[1,1] += 1
        elif y == 1 and yp == 2:
            arr[2,1] += 1
        elif y == 2 and yp == 0: 
            arr[0,2] += 1
        elif y == 2 and yp == 1:
            arr[1,2] += 1
        elif y == 2 and yp == 2:
            arr[2,2] += 1
        
    accuracy = (arr[0,0] + arr[1,1] + arr[2,2]) / ( arr.sum() )

    precision0 = arr[0,0]/(arr[0,0]+arr[0,1]+arr[0,2]) if ((arr[0,0]+arr[0,1]+arr[0,2]) != 0) else 0
    recall0 = arr[0,0]/(arr[0,0] + arr[1,0] + arr[2,0]) if ((arr[0,0] + arr[1,0] + arr[2,0]) != 0) else 0
    f1_score0 = 2*precision0*recall0 / (precision0+recall0) if ((precision0+recall0) != 0) else 0
    
    precision1 = arr[1,1]/(arr[1,0]+arr[1,1]+arr[1,2]) if ((arr[1,0]+arr[1,1]+arr[1,2]) != 0) else 0
    recall1 = arr[1,1]/(arr[1,1] + arr[0,1] + arr[2,1]) if ((arr[1,1] + arr[0,1] + arr[2,1]) != 0) else 0
    f1_score1 = 2*precision1*recall1 / (precision1+recall1) if ((precision1+recall1) != 0) else 0
    
    precision2 = arr[2,2]/(arr[2,0]+arr[2,1]+arr[2,2]) if ((arr[2,0]+arr[2,1]+arr[2,2]) != 0) else 0
    recall2 = arr[2,2]/(arr[0,2] + arr[1,2] + arr[2,2]) if ((arr[0,2] + arr[1,2] + arr[2,2]) != 0) else 0
    f1_score2 = 2*precision2*recall2 / (precision2+recall2) if ((precision2+recall2) != 0) else 0
    
    precision = [precision0, precision1, precision2]
    recall = [recall0, recall1, recall2]
    f1_score = [f1_score0, f1_score1, f1_score2]
    
    return accuracy, precision, recall, f1_score

#db_score의 feature variable
X = [ (t['homework'], t['discussion'], t['final']) for t in data ]
X = np.array(X)

#db_score의 label attribute
y = []
for t in data:
    if(t['grade'] == 'A'):
        y.append(0)
    elif(t['grade'] == 'B'):
        y.append(1)
    else: #t['grade'] = C 인 경우
        y.append(2)
y = np.array(y)

#train_test_split의 경우
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.26, random_state=42)

#SVM import

from sklearn.svm import SVC

kernelName = 'rbf'

cNum1 =5

svm = SVC(kernel = kernelName, C= int(cNum1))

svm = svm.fit(X_train, y_train)

y_predict = svm.predict(X_test)

acc, precList, recList, f1List = multi_classification_performance_eval(y_test, y_predict)

#print(X_test)

print("====================SVM multi-class classification(kernel ='"+kernelName+"', C="+str(cNum1)+") train_test_split의 경우===================")
print("train데이터 개수 : "+str(len(X_train)) )
print("test데이터 개수 : "+str(len(X_test)) )
print("accuracy=%f" %acc)
print()
print("precision(A)=%f" %precList[0])
print("recall(A)=%f" %recList[0])
print("f1_score(A)=%f" %f1List[0])
print()
print("precision(B)=%f" %precList[1])
print("recall(B)=%f" %recList[1])
print("f1_score(B)=%f" %f1List[1])
print()
print("precision(C)=%f" %precList[2])
print("recall(C)=%f" %recList[2])
print("f1_score(C)=%f" %f1List[2])

#K-fold cross validation의 경우
#db_score의 feature variable
X = [ (t['homework'], t['discussion'], t['final']) for t in data ]
X = np.array(X)

#db_score의 label attribute
y = []
for t in data:
    if(t['grade'] == 'A'):
        y.append(0)
    elif(t['grade'] == 'B'):
        y.append(1)
    else: #t['grade'] = C 인 경우
        y.append(2)
y = np.array(y)
from sklearn.model_selection import KFold

accuracy = []
A_precision = []
A_recall = []
A_f1_score = []
B_precision = []
B_recall = []
B_f1_score = []
C_precision = []
C_recall = []
C_f1_score = []

kNum2 =3  #데이트를 kNum 등분해서 교차검증
cNum2 = 50  #SVC C값

kf = KFold(n_splits= int(kNum2), random_state=42, shuffle=True)

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

for train_index, test_index in kf.split(X):
    X_train2, X_test2 = X[train_index], X[test_index]
    y_train2, y_test2 = y[train_index], y[test_index]
    
    svm2 = SVC(kernel = 'rbf' , C=int(cNum2))
    svm2 = svm2.fit(X_train2, y_train2)
    y_predict2 = svm2.predict(X_test2)

    accuracy.append(accuracy_score(y_test2, y_predict2))
    
    A_precision.append(precision_score(y_test2, y_predict2, average = None)[0])
    A_recall.append(recall_score(y_test2, y_predict2, average = None)[0])
    A_f1_score.append(f1_score(y_test2, y_predict2, average = None)[0])
    
    B_precision.append(precision_score(y_test2, y_predict2, average = None)[1])
    B_recall.append(recall_score(y_test2, y_predict2, average = None)[1])
    B_f1_score.append(f1_score(y_test2, y_predict2, average = None)[1])
    
    C_precision.append(precision_score(y_test2, y_predict2, average = None)[2])
    C_recall.append(recall_score(y_test2, y_predict2, average = None)[2])
    C_f1_score.append(f1_score(y_test2, y_predict2, average = None)[2])


   
import statistics
print()
print("====================SVM multi-classification(kernal = 'rbf', C="+str(cNum2)+") "+str(kNum2)+"-fold cross validation의 경우================")

print("average_accuracy =%f"%statistics.mean(accuracy))
print()
print("average_precision(A) =%f"%statistics.mean(A_precision))
print("average_recall(A) =%f"%statistics.mean(A_recall))
print("average_f1_score(A) =%f"%statistics.mean(A_f1_score))
print()
print("average_precision(B) =%f"%statistics.mean(B_precision))
print("average_recall(B) =%f"%statistics.mean(B_recall))
print("average_f1_score(B) =%f"%statistics.mean(B_f1_score))
print()
print("average_precision(C) =%f"%statistics.mean(C_precision))
print("average_recall(C) =%f"%statistics.mean(C_recall))
print("average_f1_score(C) =%f"%statistics.mean(C_f1_score))
print()



====================SVM multi-class classification(kernel ='rbf', C=5) train_test_split의 경우===================
train데이터 개수 : 68
test데이터 개수 : 24
accuracy=0.791667

precision(A)=0.900000
recall(A)=1.000000
f1_score(A)=0.947368

precision(B)=0.625000
recall(B)=0.714286
f1_score(B)=0.666667

precision(C)=0.833333
recall(C)=0.625000
f1_score(C)=0.714286

====================SVM multi-classification(kernal = 'rbf', C=50) 3-fold cross validation의 경우================
average_accuracy =0.695341

average_precision(A) =0.786325
average_recall(A) =0.713203
average_f1_score(A) =0.745128

average_precision(B) =0.560847
average_recall(B) =0.599327
average_f1_score(B) =0.565217

average_precision(C) =0.758333
average_recall(C) =0.767677
average_f1_score(C) =0.752513



<h1>grade A,B,C에 대한 Logistic Regression multi-class classification

In [525]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

#db_score의 feature variable
X = [ (t['homework'], t['discussion'], t['final']) for t in data ]
X = np.array(X)

#db_score의 label attribute
y = []
for t in data:
    if(t['grade'] == 'A'):
        y.append(0)
    elif(t['grade'] == 'B'):
        y.append(1)
    else: #t['grade'] = C 인 경우
        y.append(2)
y = np.array(y)

#train_test_split의 경우
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.26, random_state=42)


#스케일러 생성 => 각 feature들의 스케일을 맞추어 줘야함
#각 feature의 평균을 0, 표준편차를 1로 맞춰줌

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

cNum3 = 1

lr = LogisticRegression(C= int(cNum3),random_state=42)

lr.fit(X_train_scaled, y_train)

lr_y_predict = lr.predict(X_test_scaled)

lr_acc, lr_precList, lr_recList, lr_f1List = multi_classification_performance_eval(y_test, lr_y_predict)


print("====================Logistic Regression multi classification(C="+str(cNum3)+") train_test_split의 경우===================")
print("train데이터 개수 : "+str(len(X_train)) )
print("test데이터 개수 : "+str(len(X_test)) )
print("accuracy=%f" %lr_acc)
print()
print("precision(A)=%f" %lr_precList[0])
print("recall(A)=%f" %lr_recList[0])
print("f1_score(A)=%f" %lr_f1List[0])
print()
print("precision(B)=%f" %lr_precList[1])
print("recall(B)=%f" %lr_recList[1])
print("f1_score(B)=%f" %lr_f1List[1])
print()
print("precision(C)=%f" %lr_precList[2])
print("recall(C)=%f" %lr_recList[2])
print("f1_score(C)=%f" %lr_f1List[2])

#print('테스트 셋 관측 수 : ' + str(len(y)) )
#print('오분류 관측 수 : ', sum(lr_y_predict != y_test))

#K-fold cross validation의 경우
from sklearn.model_selection import KFold

l_accuracy = []

l_A_precision = []
l_A_recall = []
l_A_f1_score = []
l_B_precision = []
l_B_recall = []
l_B_f1_score = []
l_C_precision = []
l_C_recall = []
l_C_f1_score = []

kNum3 = 4  #데이터를 kNum 등분해서 교차검증
cNum3 = 1 #SVC C값

X = scaler.fit_transform(X)

kf = KFold(n_splits= int(kNum3), random_state=42, shuffle=True)

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

for train_index, test_index in kf.split(X):
    X_train2, X_test2 = X[train_index], X[test_index]
    y_train2, y_test2 = y[train_index], y[test_index]
    
    #X_train2_scaled = scaler.fit_transform(X_train2)
    #X_test2_scaled = scaler.fit_transform(X_test2)
    
    lr = LogisticRegression(C= int(cNum3), class_weight = 'balanced',random_state=42)
    lr.fit(X_train2, y_train2)
    lr_y_predict = lr.predict(X_test2)
    #print("===============실제============")
    #print(y_test2)
    #print("===============예측============")
    #print(lr_y_predict)
    l_accuracy.append(acc)

    accuracy.append(accuracy_score(y_test2, lr_y_predict))
    l_A_precision.append(precision_score(y_test2, lr_y_predict, average = None)[0])
    l_A_recall.append(recall_score(y_test2, lr_y_predict, average = None)[0])
    l_A_f1_score.append(f1_score(y_test2, lr_y_predict, average = None)[0])
    
    l_B_precision.append(precision_score(y_test2, lr_y_predict, average = None)[1])
    l_B_recall.append(recall_score(y_test2, lr_y_predict, average = None)[1])
    l_B_f1_score.append(f1_score(y_test2, lr_y_predict, average = None)[1])
    
    l_C_precision.append(precision_score(y_test2, lr_y_predict, average = None)[2])
    l_C_recall.append(recall_score(y_test2, lr_y_predict, average = None)[2])
    l_C_f1_score.append(f1_score(y_test2, lr_y_predict, average = None)[2])
    
import statistics
print()

print("====================Logistic Regression multi classification(C="+str(cNum3)+") "+str(kNum3)+"-fold cross validation의 경우================")
print("average_accuracy =%f"%statistics.mean(accuracy))
print()
print("average_precision of A =%f"%statistics.mean(l_A_precision))
print("average_recall of A =%f"%statistics.mean(l_A_recall))
print("average_f1_score of A =%f"%statistics.mean(l_A_f1_score))
print()

print("average_precision of B =%f"%statistics.mean(l_B_precision))
print("average_recall of B =%f"%statistics.mean(l_B_recall))
print("average_f1_score of B =%f"%statistics.mean(l_B_f1_score))
print()

print("average_precision of C =%f"%statistics.mean(l_C_precision))
print("average_recall of C =%f"%statistics.mean(l_C_recall))
print("average_f1_score of C =%f"%statistics.mean(l_C_f1_score))



====================Logistic Regression multi classification(C=1) train_test_split의 경우===================
train데이터 개수 : 68
test데이터 개수 : 24
accuracy=0.791667

precision(A)=1.000000
recall(A)=0.777778
f1_score(A)=0.875000

precision(B)=0.625000
recall(B)=0.714286
f1_score(B)=0.666667

precision(C)=0.777778
recall(C)=0.875000
f1_score(C)=0.823529

====================Logistic Regression multi classification(C=1) 4-fold cross validation의 경우================
average_accuracy =0.683096

average_precision of A =0.722403
average_recall of A =0.787302
average_f1_score of A =0.742544

average_precision of B =0.538393
average_recall of B =0.520833
average_f1_score of B =0.517666

average_precision of C =0.809091
average_recall of C =0.694444
average_f1_score of C =0.717735
